In [ ]:
from pathlib import Path
import shutil
import os
import subprocess

In [ ]:
def link_files(input_path):
    os.link(str(input_path / 'flair.nii.gz'), str(input_path / 'FLAIR.nii.gz'))
    os.link(str(input_path / 'flair.nii.gz'), str(input_path / 'fla.nii.gz'))
    os.link(str(input_path / 't1.nii.gz'), str(input_path / 'T1.nii.gz'))
    os.link(str(input_path / 't1ce.nii.gz'), str(input_path / 'T1c.nii.gz'))
    os.link(str(input_path / 't1ce.nii.gz'), str(input_path / 't1c.nii.gz'))
    os.link(str(input_path / 't2.nii.gz'), str(input_path / 'T2.nii.gz'))


def run_docker(input_path):
    command = f'docker run --runtime=nvidia -i --rm -v {input_path.absolute()}' +\
               ':/data fabianisensee/isen2018 python3.6 predict_patient.py'
    result = subprocess.run(command, shell=True, capture_output=True)
    if result.returncode != 0:
        print(result.stdout)
        print(result.stderr)
        raise Exception('failed to execute docker')

In [ ]:
def baseline(image):
    base_path = Path('images') / image

    input_path = base_path / 'input'
    if input_path.exists():
        shutil.rmtree(input_path)
    input_path.mkdir()

    for mod in ['t1', 't1ce', 't2', 'flair']:
        os.link(base_path / 'original' / f'{mod}.nii.gz',
                input_path / f'{mod}.nii.gz')

    results_path = input_path / 'results'
    results_path.mkdir()
    link_files(input_path)

    run_docker(input_path)

    result_file = list(results_path.iterdir())[0]
    os.rename(result_file, base_path / 'baseline.nii.gz')

In [ ]:
def run(image):
    base_path = Path('images') / image
    mods = set(['t1', 't1ce', 't2', 'flair'])
    for mod in mods:
        mod_path = base_path / mod

        mod_results_path = base_path / f'{mod}_results'
        if mod_results_path.exists():
            shutil.rmtree(mod_results_path)
        mod_results_path.mkdir()

        for file in mod_path.iterdir():
            print(str(file))
            input_path = base_path / 'input'
            if input_path.exists():
                shutil.rmtree(input_path)
            input_path.mkdir()

            os.link(file, input_path / f'{mod}.nii.gz')

            other_mods = mods - set([mod])
            for other_mod in other_mods:
                os.link(base_path / 'original' / f'{other_mod}.nii.gz',
                        input_path / f'{other_mod}.nii.gz')

            link_files(input_path)

            results_path = input_path / 'results'
            results_path.mkdir()

            run_docker(input_path)

            result_file = list(results_path.iterdir())[0]
            os.rename(result_file, mod_results_path / file.name)


image = 'Brats18_2013_17_1'
baseline(image)
run(image)